Para replicar el referenciamiento de ImageNet en tu propio dataset de imágenes almacenado en Amazon S3 y ejecutar la transferencia de aprendizaje con InceptionV3, debes seguir varios pasos. Aquí tienes una guía general sobre cómo hacerlo:


In [1]:
# Librerías estándar
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO

# TensorFlow y Keras
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D, MaxPooling2D, Reshape)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# Otras librerías
import os
import boto3
from PIL import Image
from sklearn.metrics import (accuracy_score, auc, average_precision_score, classification_report, 
                             confusion_matrix, precision_score, recall_score)
from sklearn.model_selection import train_test_split

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-10-12 18:32:57.653747: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Entrenamiento Animal y No Animal

In [2]:
# Carga la base de datos de Train y Test
train = pd.read_pickle('trainingAnimal.pkl')
test = pd.read_pickle('testingAnimal.pkl')

In [3]:
X_train = train['Imagen']
y_train = train['Animal']
X_test = test['Imagen']
y_test = test['Animal']

In [5]:
# Convierte los datos de entrenamiento y etiquetas en tensores de TensorFlow
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))
X_test_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_test]))

## Descarga del modelo InceptionV3 pre-entrenado:
Descarga el modelo InceptionV3 pre-entrenado con pesos de ImageNet. Puedes hacerlo utilizando TensorFlow o Keras.

In [8]:
input_shape = (150, 150, 3)

In [9]:
# Carga el modelo InceptionV3 preentrenado con pesos de ImageNet (no incluye las capas densas superiores)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

87910968/87910968 [==============================] - 9s 0us/step


## Generador de datos:

Utiliza un generador de datos de Keras para cargar y preprocesar tus imágenes desde S3. Debes proporcionar la ruta a tus imágenes en S3 y etiquetas correspondientes. Aquí un ejemplo de cómo configurar un generador de datos:

In [10]:
# Número de clases en tu conjunto de datos
num_classes = len(y_train.unique())

In [11]:
# Tamaño del lote (batch size) que deseas utilizar durante el entrenamiento
batch_size = 32

## Entrenamiento del modelo:

Añade capas personalizadas en la parte superior del modelo InceptionV3 y entrena el modelo en tus datos utilizando el generador de datos. Asegúrate de congelar las capas base de InceptionV3 para que no se actualicen durante el entrenamiento.

In [12]:
# Agregar capas personalizadas en la parte superior del modelo base
x = Flatten()(base_model.output) #Flatten output to 1 dimension
x = Dense(1024,activation='relu')(x) #Añade una layer con Relu activation
x = Dropout(0.2)(x) #Añade un dropout rate de 0.2
predictions = Dense(1, activation = 'sigmoid')(x)

In [13]:
# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

In [14]:
# Congelar las capas del modelo base para el transfer learning
for layer in base_model.layers:
    layer.trainable = False

In [16]:
# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

In [18]:
# Entrena el modelo con los datos de entrenamiento
model.fit(X_train_tf, y_train, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/30
119/119 [==============================] - 66s 518ms/step - loss: 43.1880 - accuracy: 0.8454 - val_loss: 5.8533 - val_accuracy: 0.9221
Epoch 2/30
119/119 [==============================] - 57s 480ms/step - loss: 1.9160 - accuracy: 0.9339 - val_loss: 1.8785 - val_accuracy: 0.7916
Epoch 3/30
119/119 [==============================] - 58s 488ms/step - loss: 0.3251 - accuracy: 0.9434 - val_loss: 0.2038 - val_accuracy: 0.9453
Epoch 4/30
119/119 [==============================] - 91s 763ms/step - loss: 0.1279 - accuracy: 0.9605 - val_loss: 0.2401 - val_accuracy: 0.9316
Epoch 5/30
119/119 [==============================] - 233s 2s/step - loss: 0.1045 - accuracy: 0.9663 - val_loss: 0.2160 - val_accuracy: 0.9453
Epoch 6/30
119/119 [==============================] - 294s 2s/step - loss: 0.0937 - accuracy: 0.9692 - val_loss: 0.2005 - val_accuracy: 0.9453
Epoch 7/30
119/119 [==============================] - 233s 2s/step - loss: 0.0964 - accuracy: 0.9665 - val_loss: 0.1737 - val_accurac

In [22]:
# Guardar el modelo entrenado
model.save('modeloAnimalIV3.h5')

In [19]:
y_proba = model.predict(X_test_tf)
y_pred = (y_proba >= 0.5).astype(int)

38/38 [==============================] - 34s 830ms/step


In [20]:
# Calculamos Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculamos Recall
recall = recall_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Recall: {recall}")

# Calculamos Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
print(f"Specificity: {specificity}")

Accuracy: 0.9604043807919124
Recall: 0.9854838709677419
Specificity: 0.9329805996472663


In [21]:
# Calcular average precision
ap = average_precision_score(y_test, y_proba)

print("Average Precision (AP):", ap)

Average Precision (AP): 0.966844633225649


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.95      0.97       567
           1       0.96      0.99      0.97       620

    accuracy                           0.97      1187
   macro avg       0.97      0.97      0.97      1187
weighted avg       0.97      0.97      0.97      1187



# Entrenamiento Guanaco y No Guanaco

In [65]:
# Carga la base de datos de Train y Test
train = pd.read_pickle('trainingGuanaco.pkl')
test = pd.read_pickle('testingGuanaco.pkl')

In [66]:
X_train = train['Imagen']
y_train = train['Guanaco']
X_test = test['Imagen']
y_test = test['Guanaco']

In [67]:
# Convierte los datos de entrenamiento y etiquetas en tensores de TensorFlow
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))
X_test_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_test]))

## Balanceo

In [68]:
y_train.value_counts()

Guanaco
True     1187
False     541
Name: count, dtype: int64

La base esta desbalanceada ya que existen muchos mas guanacos que otros animales. Para resolver esto se somete al resto de los animales a tecnicas de *Data Augmentation*

Data Augmentation: make training set larger by applying transformations. More information to learn from.
- Brightness and Contrast adjustments
- Rotations
- Gaussian noise
- Mirroring

In [70]:
def data_augmentation(image_tensor):

    # Convierte el tensor de imagen a una imagen TensorFlow
    image = tf.convert_to_tensor(image_tensor, dtype=tf.float32)

    # Brightness and Contrast adjustments
    if np.random.rand() < 0.8:
        image = tf.image.adjust_brightness(image, delta=0.2)  # Cambiar el brillo
        image = tf.image.adjust_contrast(image, contrast_factor=1.2)  # Cambiar el contraste

    # Rotations
    if np.random.rand() < 0.7:
        degrees = np.random.uniform(-10, 10)  # Rotación aleatoria entre -10 y 10 grados
        degrees = int(round(degrees))  # Redondea los grados a un entero
        image = tf.image.rot90(image, k=degrees // 90)  # Rotar la imagen

    # Gaussian noise
    if np.random.rand() < 0.2:
        noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.1)
        image = image + noise

    # Mirroring (flip horizontal)
    if np.random.rand() < 0.5:
        image = tf.image.flip_left_right(image)

    # Convierte la imagen aumentada de nuevo a un tensor
    augmented_image_tensor = tf.convert_to_tensor(image.numpy(), dtype=tf.float32)

    return augmented_image_tensor

In [71]:
# Crea una nueva lista para almacenar los tensores de imágenes aumentados
imagen_tensor_aumentada = []

# Itera a través de las filas del DataFrame y aplica la función de aumento de datos
for index, row in train.iterrows():
    if np.random.rand() < 0.38:
        imagen_tensor = row['Imagen']
        imagen_aumentada = data_augmentation(imagen_tensor)
        imagen_tensor_aumentada.append(imagen_aumentada)

In [72]:
train_augmentation = pd.DataFrame(columns=['Imagen', 'Guanaco'])
train_augmentation['Imagen'] = imagen_tensor_aumentada
train_augmentation['Guanaco'] = False

In [73]:
train_augmentation= pd.concat([train, train_augmentation], ignore_index=True)

In [74]:
train_augmentation['Guanaco'].value_counts()

Guanaco
False    1199
True     1187
Name: count, dtype: int64

In [75]:
X_train = train_augmentation['Imagen']
y_train = train_augmentation['Guanaco']

In [76]:
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))

## Descarga del modelo VGG16 pre-entrenado:
Descarga el modelo VGG16 pre-entrenado con pesos de ImageNet. Puedes hacerlo utilizando TensorFlow o Keras.

In [77]:
input_shape = (150, 150, 3)

In [78]:
# Carga el modelo InceptionV3 preentrenado con pesos de ImageNet (no incluye las capas densas superiores)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

## Entrenamiento del modelo:

Añade capas personalizadas en la parte superior del modelo VGG16 y entrena el modelo en tus datos utilizando el generador de datos. Asegúrate de congelar las capas base de VGG16 para que no se actualicen durante el entrenamiento.

In [79]:
# Agregar capas personalizadas en la parte superior del modelo base
x = Flatten()(base_model.output)  # Flatten output to 1 dimension
# x = Dense(1024, activation='relu')(x)  # Añade una capa con Relu activation
# x = Dropout(0.2)(x)  # Añade un dropout rate de 0.2

# # Agregar más capas densas
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.3)(x)

# # Agregar más capas densas
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.2)(x)


# Capa de salida
predictions = Dense(1, activation='sigmoid')(x)

In [80]:
# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

In [81]:
# Congelar las capas del modelo base para el transfer learning
for layer in base_model.layers:
    layer.trainable = False

In [82]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

In [83]:
# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [84]:
# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_tf, y_train, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/30
60/60 [==============================] - 92s 1s/step - loss: 6.9560 - accuracy: 0.5954 - val_loss: 5.4525 - val_accuracy: 0.4770
Epoch 2/30
60/60 [==============================] - 77s 1s/step - loss: 3.1485 - accuracy: 0.6546 - val_loss: 8.2788 - val_accuracy: 0.3619
Epoch 3/30
60/60 [==============================] - 80s 1s/step - loss: 2.4727 - accuracy: 0.6944 - val_loss: 6.6159 - val_accuracy: 0.3808
Epoch 4/30
60/60 [==============================] - 81s 1s/step - loss: 1.9776 - accuracy: 0.7144 - val_loss: 4.5897 - val_accuracy: 0.4874
Epoch 5/30
60/60 [==============================] - 83s 1s/step - loss: 1.9030 - accuracy: 0.7123 - val_loss: 3.3831 - val_accuracy: 0.5481
Epoch 6/30
60/60 [==============================] - 75s 1s/step - loss: 1.3765 - accuracy: 0.7458 - val_loss: 1.9861 - val_accuracy: 0.6987
Epoch 7/30
60/60 [==============================] - 77s 1s/step - loss: 1.4476 - accuracy: 0.7411 - val_loss: 8.4532 - val_accuracy: 0.2887
Epoch 8/30
60/60 [==

In [33]:
# Guardar el modelo entrenado
model.save('modeloGuanacoIV3.h5')

In [63]:
y_proba = model.predict(X_test_tf)
y_pred = (y_proba >= 0.5).astype(int)

14/14 [==============================] - 17s 1s/step


In [64]:
# Calculamos Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculamos Recall
recall = recall_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Recall: {recall}")

# Calculamos Precision
precision = precision_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Precision: {precision}")

# Calculamos Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
print(f"Specificity: {specificity}")

# Accuracy: 0.74364896073903
# Recall: 0.9932659932659933
# Precision: 0.7301980198019802
# Specificity: 0.19852941176470587

Accuracy: 0.6235565819861432
Recall: 0.5420875420875421
Precision: 0.8563829787234043
Specificity: 0.8014705882352942


In [54]:
# Calcular average precision
ap = average_precision_score(y_test, y_proba)

print("Average Precision (AP):", ap)

Average Precision (AP): 0.7893658695225838


In [55]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.36      0.86      0.51       138
        True       0.82      0.29      0.43       299

    accuracy                           0.47       437
   macro avg       0.59      0.58      0.47       437
weighted avg       0.68      0.47      0.46       437



In [ ]:
(y_pred == 0).sum() / len(y_pred)

0.7780320366132724

# Entrenamiento Categoria Especie

In [ ]:
# Carga la base de datos de Train y Test
train = pd.read_pickle('trainingCategoria.pkl')
test = pd.read_pickle('testingCategoria.pkl')

In [ ]:
X_train = train['Imagen']
y_train = train['Categoria']
X_test = test['Imagen']
y_test = test['Categoria']

In [ ]:
# Convierte los datos de entrenamiento y etiquetas en tensores de TensorFlow
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))
X_test_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_test]))